# PDF2KG 批量处理流程

本 notebook 支持批量处理多个 PDF 文件，构建统一的知识图谱

## 流程概览
1. 批量提取所有 PDF 文本
2. 合并文本并分块
3. 构建知识图谱
4. 可视化和存储

In [1]:
import subprocess
import time

# 检查 Docker 是否运行
try:
    subprocess.run(["docker", "ps"], capture_output=True, check=True)
    print("✓ Docker 正在运行")
except:
    print("✗ 请先启动 Docker")
    raise

import os

# 检查容器是否存在
result = subprocess.run(
    ["docker", "ps", "-a", "--filter", "name=neo4j-kg", "--format", "{{.Names}}"],
    capture_output=True, text=True
)

if "neo4j-kg" in result.stdout:
    subprocess.run(["docker", "start", "neo4j-kg"], check=True)
    print("✓ Neo4j 容器已启动")
else:
    subprocess.run([
        "docker", "run", "-d", "--name", "neo4j-kg",
        "-p", "7474:7474", "-p", "7687:7687",
        "-e", "NEO4J_AUTH=neo4j/password123",
        "neo4j:latest"
    ], check=True)
    print("✓ Neo4j 容器已创建并启动")

print("\n等待 Neo4j 启动...")
time.sleep(15)
print("✓ Neo4j 已就绪: http://localhost:7474 (neo4j/password123)")

✓ Docker 正在运行
✓ Neo4j 容器已启动

等待 Neo4j 启动...
✓ Neo4j 已就绪: http://localhost:7474 (neo4j/password123)


In [2]:
%load_ext autoreload
%autoreload 2

# 强制清除缓存
import sys
for module in list(sys.modules.keys()):
    if module.startswith('src.'):
        del sys.modules[module]

import json
from pathlib import Path
from dotenv import load_dotenv

load_dotenv()

from src.pdf_parser import extract_text_from_pdfs, chunk_text
from src.kg_builder import KGBuilder
from src.visualizer import KGVisualizer
from src.neo4j_storage import Neo4jStorage

## 2. 批量提取 PDF 文本

In [3]:
# 批量提取 data 目录下所有 PDF
pdf_texts = extract_text_from_pdfs("data")
print(f"\n共处理 {len(pdf_texts)} 个 PDF 文件")

for filename, text in pdf_texts.items():
    print(f"  - {filename}: {len(text)} 字符")

处理: 4ERGo-1762148839583-泰山松材线虫病、美国白蛾风险分析评估与预警技术研究_申卫星.pdf
处理: JItW6-1762148810582-中国森林保护学科发展历程研究_曾凡勇.pdf
处理: JU3yc-1762148851602-马尾松蛀干害虫种群动态与松材线虫病的关系及松褐天牛天敌研究_展茂魁.pdf
处理: Pc5Bj-1762148820023-松材线虫病媒介天牛种类调查及其传播机制研究_王洋.pdf
处理: XlZVN-1762148845443-浙江省松材线虫病环境影响经济评价与治理研究_李兰英.pdf
处理: ZGds6-1762148816090-松材线虫病害早期星-机-地遥感诊断光谱特征研究_李霓雯.pdf

共处理 6 个 PDF 文件
  - 4ERGo-1762148839583-泰山松材线虫病、美国白蛾风险分析评估与预警技术研究_申卫星.pdf: 88839 字符
  - JItW6-1762148810582-中国森林保护学科发展历程研究_曾凡勇.pdf: 75900 字符
  - JU3yc-1762148851602-马尾松蛀干害虫种群动态与松材线虫病的关系及松褐天牛天敌研究_展茂魁.pdf: 177606 字符
  - Pc5Bj-1762148820023-松材线虫病媒介天牛种类调查及其传播机制研究_王洋.pdf: 153195 字符
  - XlZVN-1762148845443-浙江省松材线虫病环境影响经济评价与治理研究_李兰英.pdf: 208593 字符
  - ZGds6-1762148816090-松材线虫病害早期星-机-地遥感诊断光谱特征研究_李霓雯.pdf: 132285 字符


## 3. 合并文本并分块

In [4]:
# 合并所有文本
all_text = "\n\n".join(pdf_texts.values())
print(f"合并后总长度: {len(all_text)} 字符")

# 分块
chunks = chunk_text(all_text, chunk_size=2000, overlap=200)
print(f"分成 {len(chunks)} 个块")

合并后总长度: 836428 字符
分成 465 个块


## 4. 构建知识图谱

In [5]:
kg_builder = KGBuilder(max_workers=20)
kg_data = kg_builder.build_kg_from_chunks(chunks)

print(f"\n实体数: {len(kg_data['entities'])}")
print(f"关系数: {len(kg_data['relations'])}")

构建知识图谱:  25%|██▍       | 115/465 [03:27<10:42,  1.84s/块, 实体=1390, 关系=1570]           

✗ 块 60 出错: Expecting property name enclosed in double quotes: line 188 column 45 (char 11745)


构建知识图谱:  92%|█████████▏| 430/465 [12:29<00:39,  1.12s/块, 实体=4870, 关系=6498]           

✗ 块 381 出错: Expecting ',' delimiter: line 593 column 1 (char 14450)


构建知识图谱: 100%|██████████| 465/465 [14:44<00:00,  1.90s/块, 实体=5451, 关系=7150]


实体数: 5451
关系数: 7150


In [6]:
output_dir = Path("output")
output_dir.mkdir(exist_ok=True)

# 保存 JSON
with open(output_dir / "kg_batch.json", 'w', encoding='utf-8') as f:
    json.dump(kg_data, f, ensure_ascii=False, indent=2)

# 可视化和存储
visualizer = KGVisualizer()
visualizer.build_graph(kg_data)
print(visualizer.get_stats())

# NetworkX 可视化
visualizer.visualize(output_path="output/kg_batch.png", max_nodes=100)

# 存储到 Neo4j（可选）
visualizer.store_to_neo4j(neo4j_password="password123")

{'节点数': 5451, '边数': 7150}
只显示前 100 个重要节点
✓ 图谱已保存: output/kg_batch.png
已连接到 Neo4j: bolt://localhost:7687
数据库已清空
已存储 5451 个实体和 7150 个关系
Neo4j 连接已关闭

✓ 已存储到 Neo4j
  访问: http://localhost:7474
  查询: MATCH (n)-[r]->(m) RETURN n,r,m LIMIT 100
